In [1]:
import json
from pathlib import Path

schema_path = Path("business_plan_schema.json")
with schema_path.open(encoding="utf-8") as f:
    schema = json.load(f)

In [2]:
schema_keys = list(schema.keys())
general_information = schema_keys[:5]

In [3]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# 프롬프트 정의
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "너는 전문 사업 컨설턴트이자 사업계획서 작성 전문가이다. 스타트업, 중소기업, 투자 유치용 사업계획서 작성에 특화되어 있으며, 사용자가 제공하는 정보와 요구사항을 바탕으로 체계적이고 설득력 있는 사업계획서를 작성하는 역할을 수행한다.",
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        (
            "user",
            '''스키마에서 이번에 채워야 할 필드는 **{general_information}**이다.
[필드별 작성 가이드 & 예시]
- 사업아이템명: "OO기술이 적용된 OO기능의(혜택을 제공하는) OO제품·서비스 등"
- 산출물(협약기간 내 목표): "모바일 어플리케이션(0개), 웹사이트(0개)"
- 직업(직장명 기재 불가): "교수 / 연구원 / 사무직 / 일반인 / 대학생 등"
- 기업(예정)명

[팀 구성 현황 리스트(대표자 본인 제외) - 각 행의 예시]
- 직위: "공동대표", "대리" 등
- 담당 업무: "S/W 개발 총괄", "홍보 및 마케팅" 등
- 보유 역량: "OO학 학사", "OO 관련 경력(00년 이상)"
- 구성 상태: "완료", "예정"

[작성 규칙]
1) 반드시 한국어로 작성.

[검색된 참고자료]
- 회사DB: None
- 공고문DB: None


위 정보를 참고하여 {request}'''
        ), 
    ]
)

In [4]:
from langchain_openai import ChatOpenAI

# llm = ChatOpenAI(
#     # model="gpt-5-nano"
#     model="gpt-5"
# )

llm = ChatOpenAI(
    openai_api_key="EMPTY",
    openai_api_base="http://localhost:8000/v1",
    model="K-intelligence/Midm-2.0-Base-Instruct",
)

In [5]:
from typing import List
from pydantic import BaseModel, Field

class GeneralMember(BaseModel):
    """팀 구성 현황(대표자 본인 제외)의 한 행"""
    position: str = Field(default="", description="직위")
    task: str = Field(default="", description="담당 업무")
    capability: str = Field(default="", description="보유 역량")
    status: str = Field(default="", description="구성 상태")

class GeneralInfo(BaseModel):
    """사업계획서 - 일반현황 섹션"""
    item_name: str = Field(default="", description="사업아이템명")
    deliverables: str = Field(default="", description="산출물(협약기간 내 목표)")
    job: str = Field(default="", description="직업(직장명 기재 불가)")
    company_name: str = Field(default="", description="기업(예정)명")
    general_members: List[GeneralMember] = Field(
        default_factory=list,
        description="팀 구성 현황 리스트(대표자 본인 제외)"
    )

In [6]:
structured_llm = llm.with_structured_output(GeneralInfo)

In [7]:
def to_json(model: GeneralInfo) -> str:
    return model.model_dump_json()

chain = prompt | structured_llm | to_json

In [8]:
from langchain_community.chat_message_histories import ChatMessageHistory

# 세션 기록을 저장할 딕셔너리
store = {}

# 세션 ID를 기반으로 세션 기록을 가져오는 함수
def get_session_history(session_ids):
    print(f"[대화 세션ID]: {session_ids}")
    if session_ids not in store:  # 세션 ID가 store에 없는 경우
        # 새로운 ChatMessageHistory 객체를 생성하여 store에 저장
        store[session_ids] = ChatMessageHistory()
    return store[session_ids]  # 해당 세션 ID에 대한 세션 기록 반환

In [9]:
from langchain_core.runnables.history import RunnableWithMessageHistory

chain_with_history = RunnableWithMessageHistory(
    chain,
    get_session_history,  # 세션 기록을 가져오는 함수
    input_messages_key="request",  # 사용자의 질문이 템플릿 변수에 들어갈 key
    history_messages_key="chat_history",  # 기록 메시지의 키
)

In [10]:
import uuid

session_id = uuid.uuid4()

chain_with_history.invoke(
    # 질문 입력
    {"request": f"**{general_information}** 항목을 작성하라.", "general_information": general_information},
    # 세션 ID 기준으로 대화를 기록합니다.
    config={"configurable": {"session_id": session_id}},
)

[대화 세션ID]: bc01455b-f1f2-4f3e-8f9f-a628d8cad6ae


'{"item_name":"AI 기반 한국어 비즈니스 문서 자동 분석 및 개선 서비스","deliverables":"클라우드 기반 웹 플랫폼(1개) 및 모바일 앱(1개)","job":"연구원 / 대학생","company_name":"KoreanDocOptimizer Co., Ltd.","general_members":[{"position":"공동창업자","task":"서비스 기획 및 전략 수립","capability":"경영학 학사, IT 스타트업 경험 3년","status":"완료"},{"position":"개발팀장","task":"백엔드 시스템 개발","capability":"컴퓨터공학 석사, 소프트웨어 개발 경력 7년","status":"예정"}]}'